In [11]:
import sys
import re

# Global counter for generating fresh variable names for alpha conversion
fresh_var_counter = 0
def fresh_var(base):
    global fresh_var_counter
    fresh_var_counter += 1
    return f"{base}_{fresh_var_counter}"

# Abstract Syntax Tree (AST) Node Definitions

class Term:
    def free_vars(self):
        raise NotImplementedError

    def substitute(self, var, value):
        raise NotImplementedError

    def __str__(self):
        raise NotImplementedError

class Variable(Term):
    def __init__(self, name):
        self.name = name

    def free_vars(self):
        return {self.name}

    def substitute(self, var, value):
        if self.name == var:
            return value
        else:
            return self

    def __str__(self):
        return self.name

class Abstraction(Term):
    def __init__(self, param, body):
        self.param = param
        self.body = body

    def free_vars(self):
        # Free variables of the abstraction are those free in the body, minus the bound parameter.
        return self.body.free_vars() - {self.param}

    def substitute(self, var, value):
        if self.param == var:
            return self
        elif self.param in value.free_vars():
            # To avoid variable capture, perform alpha conversion.
            new_param = fresh_var(self.param)
            new_body = self.body.substitute(self.param, Variable(new_param))
            return Abstraction(new_param, new_body.substitute(var, value))
        else:
            return Abstraction(self.param, self.body.substitute(var, value))

    def __str__(self):
        # Using the λ notation for abstraction.
        return f"(λ{self.param}.{self.body})"

class Application(Term):
    def __init__(self, func, arg):
        self.func = func
        self.arg = arg

    def free_vars(self):
        # Free variables are the union of free variables in both the function and the argument.
        return self.func.free_vars() | self.arg.free_vars()

    def substitute(self, var, value):
        return Application(self.func.substitute(var, value),
                           self.arg.substitute(var, value))

    def __str__(self):
        return f"({self.func} {self.arg})"

# Lexer and Parser for Lambda Expressions

# Regular expression to tokenize input. This pattern matches:
#  - Left parentheses: "("
#  - Right parentheses: ")"
#  - Dot: "."
#  - Lambda symbol: "\" or "λ"
#  - Identifiers: sequences of letters, digits, or underscores starting with a letter or underscore.
token_pattern = re.compile(r'\s*(?:(\()|(\))|(\.)|(\\|λ)|([A-Za-z_][A-Za-z0-9_]*))')

def tokenize(s):
    tokens = []
    pos = 0
    while pos < len(s):
        match = token_pattern.match(s, pos)
        if not match:
            raise SyntaxError(f"Unexpected character at position {pos}: {s[pos]}")
        if match.group(1):
            tokens.append(('LPAREN', '('))
        elif match.group(2):
            tokens.append(('RPAREN', ')'))
        elif match.group(3):
            tokens.append(('DOT', '.'))
        elif match.group(4):
            tokens.append(('LAMBDA', 'λ'))
        elif match.group(5):
            tokens.append(('IDENT', match.group(5)))
        pos = match.end()
    return tokens

class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0

    def peek(self):
        if self.pos < len(self.tokens):
            return self.tokens[self.pos]
        return None

    def consume(self, expected_type=None):
        token = self.peek()
        if token is None:
            return None
        if expected_type and token[0] != expected_type:
            raise SyntaxError(f"Expected token type {expected_type}, got {token}")
        self.pos += 1
        return token

    def parse(self):
        term = self.parse_expr()
        if self.pos != len(self.tokens):
            raise SyntaxError("Extra tokens at end of input")
        return term

    def parse_expr(self):
        # An expression may be an abstraction or an application.
        token = self.peek()
        if token and token[0] == 'LAMBDA':
            return self.parse_abstraction()
        else:
            return self.parse_application()

    def parse_abstraction(self):
        # Abstraction syntax: LAMBDA IDENT+ DOT expr
        self.consume('LAMBDA')
        params = []
        while True:
            token = self.peek()
            if token is None or token[0] != 'IDENT':
                break
            params.append(self.consume('IDENT')[1])
        if not params:
            raise SyntaxError("Expected at least one parameter in abstraction")
        if self.peek() is None or self.peek()[0] != 'DOT':
            raise SyntaxError("Expected '.' after parameters in abstraction")
        self.consume('DOT')
        body = self.parse_expr()
        # Convert multi-parameter abstractions into nested abstractions.
        for param in reversed(params):
            body = Abstraction(param, body)
        return body

    def parse_application(self):
        # Parse one or more atoms and treat them as left-associative applications.
        term = self.parse_atom()
        while True:
            token = self.peek()
            if token is None or token[0] in ('RPAREN', 'DOT'):
                break
            next_atom = self.parse_atom()
            term = Application(term, next_atom)
        return term

    def parse_atom(self):
        token = self.peek()
        if token is None:
            raise SyntaxError("Unexpected end of input")
        if token[0] == 'IDENT':
            return Variable(self.consume('IDENT')[1])
        elif token[0] == 'LPAREN':
            self.consume('LPAREN')
            term = self.parse_expr()
            if self.peek() is None or self.peek()[0] != 'RPAREN':
                raise SyntaxError("Expected ')'")
            self.consume('RPAREN')
            return term
        elif token[0] == 'LAMBDA':
            # Allow an abstraction to appear where an atom is expected.
            return self.parse_abstraction()
        else:
            raise SyntaxError(f"Unexpected token: {token}")

def parse_lambda(s):
    tokens = tokenize(s)
    parser = Parser(tokens)
    return parser.parse()

# Reduction Functions

def reduce_term(term, max_steps=1000):
    """Reduce a lambda term using beta reduction until no more reductions can be made
       or until a step limit is reached (to avoid non-termination)."""
    current = term
    for _ in range(max_steps):
        new_term, reduced = reduce_once(current)
        if not reduced:
            return new_term
        current = new_term
    print("Reduction did not terminate within step limit.")
    return current

def reduce_once(term):
    """Perform one reduction step on the term. Returns a tuple (new_term, reduction_occurred)."""
    if isinstance(term, Application):
        if isinstance(term.func, Abstraction):
            # Beta reduction: ((λx.M) N) → M[x := N]
            return (term.func.body.substitute(term.func.param, term.arg), True)
        else:
            # Try reducing the function part.
            new_func, reduced = reduce_once(term.func)
            if reduced:
                return (Application(new_func, term.arg), True)
            else:
                # Try reducing the argument.
                new_arg, reduced = reduce_once(term.arg)
                if reduced:
                    return (Application(term.func, new_arg), True)
                else:
                    return (term, False)
    elif isinstance(term, Abstraction):
        new_body, reduced = reduce_once(term.body)
        if reduced:
            return (Abstraction(term.param, new_body), True)
        else:
            return (term, False)
    else:
        # No reduction possible.
        return (term, False)


def perform():
    print("Lambda Calculus Interpreter")
    print("Enter lambda expressions using '\\' or 'λ' for abstraction.")
    print("For example: (\\x. x) y")
    print("Type 'quit' to exit.")
    while True:
        try:
            s = input("λ> ")
        except EOFError:
            break
        if s.strip() == "quit":
            break
        if s.strip() == "":
            continue
        try:
            term = parse_lambda(s)
            print("Parsed term:", term)
            reduced = reduce_term(term)
            print("Reduced term:", reduced)
        except Exception as e:
            print("Error:", e)



In [12]:
import os

if len(sys.argv) > 1 and os.path.isfile(sys.argv[1]):
    with open(sys.argv[1], 'r') as f:
        s = f.read()
    try:
        term = parse_lambda(s)
        print("Parsed term:", term)
        reduced = reduce_term(term)
        print("Reduced term:", reduced)
    except Exception as e:
        print("Error:", e)
else:
    perform()


Lambda Calculus Interpreter
Enter lambda expressions using '\' or 'λ' for abstraction.
For example: (\x. x) y
Type 'quit' to exit.
λ> λx.x
Parsed term: (λx.x)
Reduced term: (λx.x)
λ> (λx.x) y
Parsed term: ((λx.x) y)
Reduced term: y
λ> (λx. x x) (λy. y)
Parsed term: ((λx.(x x)) (λy.y))
Reduced term: (λy.y)
λ> (λx. x) ((λy. y) z)
Parsed term: ((λx.x) ((λy.y) z))
Reduced term: z
λ> (λn. λf. λx. f (n f x)) λf. λx. f x
Parsed term: ((λn.(λf.(λx.(f ((n f) x))))) (λf.(λx.(f x))))
Reduced term: (λf.(λx.(f (f x))))
λ> (λp. λq. p q p) (λx. λy. x) (λx. λy. y)
Parsed term: (((λp.(λq.((p q) p))) (λx.(λy.x))) (λx.(λy.y)))
Reduced term: (λx.(λy.y))
λ> quit
